<a href="https://colab.research.google.com/github/R4hulR/BreastCancerNN/blob/main/BreastCancerNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


**Data Collection and Preprocessing**

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(455, 30)
(114, 30)


In [ ]:
#standardize the data
scalar = StandardScaler()
X_train= scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [ ]:
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)

**Neural Network Architecture**

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
    super(NeuralNet,self).__init__()
    # Input Layer: Connects to the first hidden layer (self.fc1)
    self.fc1 = nn.Linear(input_dim,hidden_dim)
    # Hidden Layer: The output of self.fc1 is passed through the ReLU activation function (self.fc2) and then to the second linear layer (self.fc3).
    self.fc2 = nn.ReLU()
    self.fc3 = nn.Linear(hidden_dim,output_dim)
    # Output Layer: The output of self.fc3 is passed through the sigmoid function (self.sigmoid) to produce the final output.
    self.sigmoid = nn.Sigmoid()
  def forward(self,x):
    out = self.fc1(x)
    out = self.fc2(out)
    out = self.fc3(out)
    out = self.sigmoid(out)
    return out

In [ ]:
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
epochs = 100

In [ ]:
model = NeuralNet(input_size,hidden_size,output_size).to(device)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

**Training the neural network**

In [ ]:
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train.view(-1, 1))
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        predicted = y_pred.round()
        accuracy = (predicted == y_train.view(-1, 1)).float().sum().item() / y_train.size(0)

    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy * 100:.4f}%")


Epoch 1/100, Loss: 0.1186, Accuracy: 97.8022%
Epoch 11/100, Loss: 0.1095, Accuracy: 97.8022%
Epoch 21/100, Loss: 0.1020, Accuracy: 97.8022%
Epoch 31/100, Loss: 0.0956, Accuracy: 97.8022%
Epoch 41/100, Loss: 0.0902, Accuracy: 98.0220%
Epoch 51/100, Loss: 0.0855, Accuracy: 98.2418%
Epoch 61/100, Loss: 0.0813, Accuracy: 98.4615%
Epoch 71/100, Loss: 0.0775, Accuracy: 98.4615%
Epoch 81/100, Loss: 0.0741, Accuracy: 98.2418%
Epoch 91/100, Loss: 0.0711, Accuracy: 98.2418%


**Model Evaluation**

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    predicted = y_pred.round()
    accuracy = (predicted == y_test.view(-1, 1)).float().sum().item() / y_test.size(0)
    print(f"Test Accuracy: {accuracy * 100:.4f}%")

Test Accuracy: 96.4912%
